<a href="https://colab.research.google.com/github/abdoulsn/EDA-Pima_Diabetes-NeuralNetwork/blob/master/V0_MNIST_MKT_MST_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
import datetime, os
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import datetime

!wget https://github.com/camsovangauthier/coursesNN/raw/main/Fashion_MNIST_5classes_augmentation.h5
!wget https://github.com/gaudel/NN/raw/main/pretrained_model.h5

# Usefull functions

In [ ]:
def build_CNN(input_dim, output_dim, lr=0.001):
    ''' Docstring
    input_dim: le dim de la data en input
    output_dim: la derniere couche qui doit correspondre à 2 (on a 2 type de fashions)
    lr: taux de learning rate
    '''
    model = Sequential()
    #Q1: Compléter ici
    model.add(Conv2D(32, (3, 3), input_shape=input_dim, name="couche1"))
    model.add(MaxPooling2D((2,2)),)
    model.add(Dropout(0.1))
    model.add(Conv2D(64, (3, 3),activation='relu', name="couche2"))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(64, (3, 3),activation='relu', name="couche3"))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.1))
    model.add(Flatten(name="flatten"))
    model.add(Dense(64, activation='relu',name="couche4" )) 
    model.add(Dropout(0.3)) #pour désactiver les neurones non significatifs (30 %), allège le réseau de neurones,évite le sur-apprentissage
    model.add(Dense(output_dim, activation='softmax', name="couche_sftmx"))

    #Q1: Compilation du modèle
    opt = RMSprop(learning_rate=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def build_CNN_from_pretrained(file_name, output_dim, lr=0.01):
    # Q4: compléter l'appel à la fonction load_model()
    model = load_model(file_name)
    # Q4: retirer et ajouter la/les couches nécessaires ici
    model.pop()
    model.pop()
    model.add(Dense(2, activation='softmax', name="couche_sftmx"))

    # Q4: Compilation du modèle
    return model
build_CNN_from_pretrained("/content/Fashion_MNIST_5classes_augmentation.h5", output_dim=(150,150,3)).summary()

# Main

## Hyper-parameters

In [ ]:
epochs = 50
batch_size = 64

## Initialization

In [ ]:
rng = np.random.default_rng()

## Data

### Download data

In [ ]:
data_f = get_file('data_fashion.zip', 'https://github.com/camsovangauthier/coursesNN/raw/main/data_fashion.zip', extract=True)

def foo(data_src):
  if type(data_src) == str:
    images, labels = next(ImageDataGenerator(rescale=1./255).flow_from_directory(data_f[:-4] + '/' + data_src, batch_size=25))
  else:
    images, labels = next(data_src)
  print(f'shape of features: {images.shape[1:]}')
  print(f'shape of labels: {labels.shape[1:]}')
  class_names = ['Bag', 'Dress']
  plt.figure(figsize=(10,10))
  for i in range(25):
      plt.subplot(5,5,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(images[i], cmap=plt.cm.binary)
      plt.xlabel(class_names[np.argmax(labels[i])] + f' ({labels[i]})')
  plt.show()

# print('--- train data ---')
# foo('train')

# print('--- validation data ---')
# foo('valid')

### Data generators

In [ ]:
#Q2: Ajouter l'attribut rescale aux générateurs
#Q3: Ajouter des attributs pour faire de l'augmeentation de données
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


#Q2: Ajouter les attributs "target_size" et "batch_size"
train_flow = train_datagen.flow_from_directory(
        data_f[:-4]+'/train',                   # Source de la train
        target_size=(150, 150),                 # toutes les images retaillée à 150x150
        class_mode='categorical')               # Pour le binary_crossentropy loss labels

valid_flow = test_datagen.flow_from_directory(
        data_f[:-4]+'/valid',
        target_size=(150, 150),
        class_mode='categorical')

print('--- train data ---')
foo(train_flow)

print('--- validation data ---')
foo(valid_flow)

## Build

In [ ]:
from_scratch = True
if from_scratch:
  # Q2: compléter l'appel à la fonction build_CNN()
  cnn_model = build_CNN(input_dim=train_flow.image_shape, output_dim=2,lr=0.001)
  label = 'from_scratch'
else:
  cnn_model = build_CNN_from_pretrained("/content/Fashion_MNIST_5classes_augmentation.h5", output_dim=(150,150,3))
  label = 'from_pretrained'


cnn_model.summary()

## Fit & evaluate

In [ ]:
#Q2: compléter l'appel aux fonctions fit() et evaluate()
tensorboard_callback = TensorBoard(log_dir='logs/' + label + '__' + datetime.datetime.now().strftime("%d-%m_%Hh%M"),
                                   histogram_freq=int(epochs/10))

cnn_model.fit_generator(train_flow, validation_data = valid_flow, epochs=20)

cnn_model.evaluate(verbose=2)

# Tensorboard

## Initialization

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Start tensorboard

In [ ]:
%tensorboard --logdir logs